In [1]:
with open ("requirements.txt", "w") as f:
    f.write("streamlit\npandas")

In [2]:
import streamlit as st
import pandas as pd
import sqlite3
from datetime import datetime

# Mobile-friendly layout
st.set_page_config(page_title="TradeLog Pro", layout="centered")

# Database connection
def get_connection():
    return sqlite3.connect('trades_cloud.db', check_same_thread=False)

conn = get_connection()
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS trades 
             (ticker TEXT, leverage TEXT, account TEXT, direction TEXT, 
              entry_p REAL, exit_p REAL, status TEXT, pnl REAL, reason TEXT, date TEXT)''')
conn.commit()

st.title("📱 AlphaLog Mobile")

# Log Trade Section
with st.expander("📝 Log New Trade", expanded=True):
    with st.form("mobile_form", clear_on_submit=True):
        ticker = st.text_input("Ticker (e.g., BTC)").upper()
        
        col1, col2 = st.columns(2)
        direction = col1.selectbox("Side", ["Long", "Short"])
        acc_type = col2.selectbox("Account", ["Real", "Demo"])
        
        en = st.number_input("Entry Price", step=0.00001, format="%.5f")
        ex = st.number_input("Exit Price", step=0.00001, format="%.5f")
        
        lev = st.text_input("Leverage", "10x")
        status = st.selectbox("Result", ["Win", "Loss", "BE", "Open"])
        why = st.text_area("Why was this taken?")
        
        if st.form_submit_button("Submit Trade"):
            pnl = (ex - en) if direction == "Long" else (en - ex)
            c.execute("INSERT INTO trades VALUES (?,?,?,?,?,?,?,?,?,?)",
                       (ticker, lev, acc_type, direction, en, ex, status, round(pnl, 2), why, datetime.now().strftime("%Y-%m-%d")))
            conn.commit()
            st.success("Trade Saved!")

# Stats & History
df = pd.read_sql_query("SELECT * FROM trades ORDER BY date DESC", conn)
if not df.empty:
    st.metric("Total PnL", f"${df['pnl'].sum():,.2f}")
    st.subheader("History")
    # Show simplified table for small screens
    st.dataframe(df[['ticker', 'direction', 'pnl', 'status']], use_container_width=True)

2026-01-02 23:27:24.064 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 23:27:24.082 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 23:27:26.235 
  command:

    streamlit run D:\Anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-01-02 23:27:26.237 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 23:27:26.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 23:27:26.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-02 23:27:26.248 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare m